In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

pd.set_option('display.max_columns', None)

import warnings
warnings.filterwarnings('ignore')

In [70]:
pff = pd.read_csv('pff_pi.csv', index_col=0)
pff = pff.reset_index().rename(columns={'index':'game_id'}).set_index('game_id')
pff = pff[['Team', 'Opposing Team', 'Season', 'Week', 'Date', 'Time', 'H/A', 'W/L',
       'PA', 'PF', 'OVR', 'OFF', 'PASS', 'PBLK', 'RECV', 'RUN', 'RBLK', 'DEF',
       'RDEF', 'TACK', 'PRSH', 'COV', 'SPEC', 'OVR_opp', 'OFF_opp', 'PASS_opp',
       'PBLK_opp', 'RECV_opp', 'RUN_opp', 'RBLK_opp', 'DEF_opp', 'RDEF_opp',
       'TACK_opp', 'PRSH_opp', 'COV_opp', 'SPEC_opp']]
# pff.iloc[:,9:].drop(columns=['SPEC','SPEC_opp'])
pff.tail(2)

,Team,Opposing Team,Season,Week,Date,Time,H/A,W/L,PA,PF,OVR,OFF,PASS,PBLK,RECV,RUN,RBLK,DEF,RDEF,TACK,PRSH,COV,SPEC,OVR_opp,OFF_opp,PASS_opp,PBLK_opp,RECV_opp,RUN_opp,RBLK_opp,DEF_opp,RDEF_opp,TACK_opp,PRSH_opp,COV_opp,SPEC_opp
game_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
7166,San Francisco 49Ers,Seattle Seahawks,2019,17,12/29/2019,8:20pm,A,W,21,26,79.9,88.9,88.3,81.6,84.6,72.2,69.2,64.7,65.6,66.5,58.7,65.5,56.3,60.3,70.3,67.1,76.2,67.5,58.9,68.1,39.7,37.0,26.6,61.1,42.6,66.9
7167,Seattle Seahawks,San Francisco 49Ers,2019,17,12/29/2019,8:20pm,H,L,26,21,60.3,70.3,67.1,76.2,67.5,58.9,68.1,39.7,37.0,26.6,61.1,42.6,66.9,79.9,88.9,88.3,81.6,84.6,72.2,69.2,64.7,65.6,66.5,58.7,65.5,56.3


In [71]:
df = pff.copy()

# Basic regression by single season holdout

In [72]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [73]:
# initialize lists to store predictions
test_score_predictions = []
holdout_score_predictions = []
test_game_ids = []
holdout_game_ids = []

# start at 2012 season
current_season = 2012

# loop through seasons
while current_season < 2020:
    print(current_season)

    # create train/test and holdout dataframes dataframes
    prediction_df = df[df['Season']<current_season].iloc[:,9:].drop(columns=['SPEC','SPEC_opp']).tail(1280)
    holdout_df = df[df['Season']==current_season].iloc[:,9:].drop(columns=['SPEC','SPEC_opp'])

    # split y, X arrays
    y = prediction_df.iloc[:,0]
    X = prediction_df.iloc[:,1:]

    # train/test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False, random_state=42)

    # append game_ids from y_test
    test_game_ids.append(y_test.index.values)

    # train model - add cross validate later
    reg = LinearRegression().fit(X_train, y_train)

    # predict on test set
    test_preds = reg.predict(X_test)
    # append test_preds
    test_score_predictions.append(test_preds)

    # grab actual scores from the holdout set
    actual_scores = holdout_df.iloc[:,0]
    # get holdout X dataframe
    X_holdout = holdout_df.iloc[:,1:]

    # predict on holdout set
    holdout_preds = reg.predict(X_holdout)
    # append holdout predictions
    holdout_score_predictions.append(holdout_preds)
    
    # append game_ids from holdout set
    holdout_game_ids.append(holdout_df.index.values)

    current_season+=1

2012
2013
2014
2015
2016
2017
2018
2019


In [76]:
# flatten list of lists
flat_holdout_score_predictions = [item for sublist in holdout_score_predictions for item in sublist]
flat_holdout_game_ids = [item for sublist in holdout_game_ids for item in sublist]

results = pd.DataFrame(data=flat_holdout_score_predictions, index=flat_holdout_game_ids, columns=['predicted_score'])
results_merge = results.merge(df, left_index=True, right_index=True)
results_merge.head(2)

,predicted_score,Team,Opposing Team,Season,Week,Date,Time,H/A,W/L,PA,PF,OVR,OFF,PASS,PBLK,RECV,RUN,RBLK,DEF,RDEF,TACK,PRSH,COV,SPEC,OVR_opp,OFF_opp,PASS_opp,PBLK_opp,RECV_opp,RUN_opp,RBLK_opp,DEF_opp,RDEF_opp,TACK_opp,PRSH_opp,COV_opp,SPEC_opp
3072,29.544820,Dallas Cowboys,New York Giants,2012,1,09/05/2012,8:30pm,A,W,17,24,72.1,75.2,84.9,64.1,77.9,68.0,61.5,62.9,74.4,57.9,72.0,50.9,-,64.8,68.9,83.0,49.4,68.1,50.9,61.1,57.9,70.1,89.0,64.5,44.6,-
3073,24.440888,New York Giants,Dallas Cowboys,2012,1,09/05/2012,8:30pm,H,L,24,17,64.8,68.9,83.0,49.4,68.1,50.9,61.1,57.9,70.1,89.0,64.5,44.6,-,72.1,75.2,84.9,64.1,77.9,68.0,61.5,62.9,74.4,57.9,72.0,50.9,-


In [77]:
from sklearn.metrics import mean_squared_error
math.sqrt(mean_squared_error(results_merge['predicted_score'].values, results_merge['PF'].values))

6.979726384813147